Execute WPS icclim processes with PHYTHON 

In [1]:
from owslib.wps import WebProcessingService, monitorExecution
wps = WebProcessingService(url="http://localhost:8093/wps", verbose=False, skip_caps=False)

Explore some informations:

In [2]:
wps.identification.title

'Flyingpigeon'

In [3]:
for process in wps.processes:
    process.identifier, process.title

In [4]:
for process in wps.processes:
    print '%s \t : %s '% (process.identifier, process.abstract) 

visualisation 	 : Just testing a nice script to visualise some variables 
sdm 	 : Species distribution model (SDM) 
WClass 	 : Weather Regimes based on pressure patterns (kmean method) 
extractpoints 	 : Extract Timeseries for specified coordinates from grid data 
segetalflora 	 : Species biodiversity of segetal flora. Imput files: variable:tas , domain: EUR-11 or EUR-44 
indices_single 	 : This process calculates climate indices based on one single variable. 
indices_percentile 	 : This process calculates climate indices based on one single variable and based on percentils of a referece period. 
subset_countries 	 : This process returns only the given polygon from input netCDF files. 
eobs_to_cordex 	 : downloads EOBS data in adaped CORDEX format 
ensembleRobustness 	 : Calculates the robustness as the ratio of noise to signal in an ensemle of timeseries 
analogs 	 : Search for days with analog pressure pattern 
fetch 	 : This process downloads resources (limited to 50GB)             

In [31]:
from os.path import join
path = '/home/nils/data/EUR-44'
files = ['tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19710101-19751231.nc',
            'tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19760101-19801231.nc',
            'tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19810101-19851231.nc',
            'tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19860101-19901231.nc',
            'tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19910101-19951231.nc',
            'tas_EUR-44_ICHEC-EC-EARTH_historical_r12i1p1_SMHI-RCA4_v1_day_19960101-20001231.nc',]
resource = [ join(path,nc) for nc in files ]

In [56]:
from flyingpigeon import indices #import calc_indice_percentile as cip
reload(indices)

<module 'flyingpigeon.indices' from '/home/nils/birdhouse/flyingpigeon/flyingpigeon/indices.py'>

In [65]:
from ocgis.contrib.library_icclim import IcclimTG90p
from flyingpigeon.ocgis_module import call
from flyingpigeon.utils import get_values, get_time
    
years = range(1971, 2001)
month = [12,1,2] # grouping to month
time_region = {'year': years , 'month': month} 
    
nc_ref_vals = call(resource=resource, variable='tas',  prefix='nc_ref_vals', time_region=time_region, output_format='nc')

start ocgis module


In [ ]:
from flyingpigeon.utils import get_values, get_time

arr = get_values(nc_ref_vals)
dt_arr = get_time(nc_ref_vals)
percentile = 90
window_width = 5

arr = np.ma.masked_array(arr)
dt_arr = np.ma.masked_array(dt_arr)


In [55]:
percentile_dict = IcclimTG90p.get_percentile_dict(arr, dt_arr, percentile, window_width)

In [63]:
nc_indices =  call(resource=resource, variable='tas',  prefix='indices', calc=calc, calc_grouping=calc_grouping, output_format='nc')

start ocgis module


In [ ]:
import numpy as np

In [64]:
dt_arr

array([datetime.datetime(1971, 1, 1, 12, 0),
       datetime.datetime(1971, 1, 2, 12, 0),
       datetime.datetime(1971, 1, 3, 12, 0),
       datetime.datetime(1971, 1, 4, 12, 0),
       datetime.datetime(1971, 1, 5, 12, 0),
       datetime.datetime(1971, 1, 6, 12, 0),
       datetime.datetime(1971, 1, 7, 12, 0),
       datetime.datetime(1971, 1, 8, 12, 0),
       datetime.datetime(1971, 1, 9, 12, 0),
       datetime.datetime(1971, 1, 10, 12, 0),
       datetime.datetime(1971, 1, 11, 12, 0),
       datetime.datetime(1971, 1, 12, 12, 0),
       datetime.datetime(1971, 1, 13, 12, 0),
       datetime.datetime(1971, 1, 14, 12, 0),
       datetime.datetime(1971, 1, 15, 12, 0),
       datetime.datetime(1971, 1, 16, 12, 0),
       datetime.datetime(1971, 1, 17, 12, 0),
       datetime.datetime(1971, 1, 18, 12, 0),
       datetime.datetime(1971, 1, 19, 12, 0),
       datetime.datetime(1971, 1, 20, 12, 0),
       datetime.datetime(1971, 1, 21, 12, 0),
       datetime.datetime(1971, 1, 22, 12, 0

In [62]:
#np.ma.masked_array?


reload(indices)

indices.calc_indice_percentile(resource=resource, variable=None, prefix=None, indices='TG90p', period=None, groupings='yr')

start ocgis module
start ocgis module


KeyError: (3, 1)

In [60]:
import icclim 

from ocgis import RequestDataset, OcgOperations
from ocgis.contrib.library_icclim import IcclimTG90p

########################################################################################################################
# Compute a custom percentile basis using ICCLIM. ######################################################################
########################################################################################################################

# Path to CF climate dataset. This examples uses the same file for indice and percentile basis calculation.
in_file = '/path/to/cf_data.nc'

# Subset the input dataset to return the desired base period for the percentile basis.
variable = 'tas'
years = range(1971, 2001)
time_region = {'year': years}
rd = RequestDataset(uri=resource, variable=variable)
field = rd.get()
field.get_time_region(time_region)

# Calculate the percentile basis. The data values must be a three-dimensional array.
arr = field.variables[variable].value.squeeze()
dt_arr = field.temporal.value_datetime
percentile = 90
window_width = 5
percentile_dict = IcclimTG90p.get_percentile_dict(arr, dt_arr, percentile, window_width)

In [61]:
calc = [{'func': 'icclim_TG90p', 'name': 'TG90p', 'kwds': {'percentile_dict': percentile_dict}}]
calc_grouping = 'month'
ops = OcgOperations(dataset=rd, calc=calc, calc_grouping=calc_grouping)
coll = ops.execute()

In [ ]:
execute = wps.execute(
    identifier="indices_percentile",
    inputs=[
    ('indices','TG90p'),
    ('groupings','yr'),
    ('resource',resource[0]),
    ('resource',resource[1]),
    ('resource',resource[2]),
    ('resource',resource[3]),
    ('resource',resource[4]),
    ('resource',resource[5]),
    ])

# check process if completed ...
monitorExecution(execute, sleepSecs=5)

o = execute.processOutputs[0]
print o.reference